# Laboratorio 3
## Integrantes

- Bravo Darlyn
- Torrejón Joel
- Valle Brandon
- Vega Giovanni

## 1. Objetivo General  
Entrenar y comparar el desempeño de modelos RNN, LSTM y Transformer en una tarea real de clasificación de sentimientos sobre texto.

## 2. Recursos  
Dataset de IMDb Movie Reviews:
50,000 reseñas de películas etiquetadas como positiva o negativa.  
https://huggingface.co/datasets/stanfordnlp/imdb.  
Instalen la librerías necesarias de huggingface.

In [1]:
!pip install -U datasets==2.14.5

In [2]:
from datasets import load_dataset
ds = load_dataset("stanfordnlp/imdb")

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
!pip install nltk

In [5]:
# Librerías estándar
import re
import string
from collections import Counter

# Ciencia de datos y visualización
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Procesamiento de lenguaje natural
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## 3.1  Preprocesamiento de datos
Para esta etapa puede usar la siguiente secuencia. Que no es la definitiva si usted cree que puede adicionar algún otro tratamiento, bienvenido.

* Cargar el texto sin procesar.
* Dividir en tokens.
* Convertir a minúsculas.
* Eliminar la puntuación de cada token.
* Filtrar los tokens restantes que no sean alfabéticos.  

Dividir datos en entrenamiento y prueba. O también puede usar los datos de entrenamiento validación que ya dispone el dataset


In [9]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub('', w) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    return words

In [10]:
train_data = ds['train']
test_data = ds['test']

## 3.2 RNN simple  
Entrenar y evaluar usando la librería provista por Pytorch:  
nn.RNN(embedding_dim, hidden_dim, ...)  


In [11]:
!pip install torch

In [12]:
!pip install --upgrade typing_extensions

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [14]:
# Preprocesar texto y construir vocabulario
def build_vocab(dataset, tokenizer, max_vocab_size=10000):
    counter = Counter()
    for sample in dataset:
        tokens = tokenizer(sample['text'])
        counter.update(tokens)
    most_common = counter.most_common(max_vocab_size - 2)
    vocab = {'<PAD>': 0, '<UNK>': 1}
    vocab.update({word: i+2 for i, (word, _) in enumerate(most_common)})
    return vocab

vocab = build_vocab(train_data, preprocess_text)

In [15]:
# Codificar textos y etiquetas
def encode_text(tokens, vocab, max_len=200):
    encoded = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    return encoded[:max_len] + [vocab['<PAD>']] * (max_len - len(encoded))

def encode_dataset(dataset, vocab, max_len=200):
    encoded_texts = [encode_text(preprocess_text(sample['text']), vocab, max_len) for sample in dataset]
    labels = [sample['label'] for sample in dataset]
    return torch.tensor(encoded_texts), torch.tensor(labels)

X_train, y_train = encode_dataset(train_data, vocab)
X_test, y_test = encode_dataset(test_data, vocab)

In [16]:
# Crear DataLoaders
from torch.utils.data import DataLoader, TensorDataset
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64)

In [17]:
# Definir el modelo con nn.RNN
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))

# Instanciar modelo
model = RNNClassifier(vocab_size=len(vocab), embed_dim=100, hidden_dim=128, output_dim=2)

In [18]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train_model(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Entrenar por algunas épocas
for epoch in range(5):
    loss = train_model(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

Epoch 1, Loss: 0.6966
Epoch 2, Loss: 0.6888
Epoch 3, Loss: 0.6856
Epoch 4, Loss: 0.6960
Epoch 5, Loss: 0.6960


In [19]:
from sklearn.metrics import f1_score

def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    f1 = f1_score(all_labels, all_preds, average='weighted')  # otros parametros 'macro' o 'micro'

    return accuracy, f1

# Evaluación
accuracy, f1 = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.4991
F1 Score: 0.4879


## 3.3 LSTM
nn.LSTM(embedding_dim, hidden_dim, ...)


In [20]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)                # (batch, seq_len, embed_dim)
        output, (hidden, cell) = self.lstm(embedded)
        return self.fc(hidden.squeeze(0))           # (batch, output_dim)


In [21]:
model = LSTMClassifier(vocab_size=len(vocab), embedding_dim=100, hidden_dim=128, output_dim=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
for epoch in range(5):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.6925
Epoch 2, Loss: 0.6762
Epoch 3, Loss: 0.6507
Epoch 4, Loss: 0.6560
Epoch 5, Loss: 0.6053


In [23]:
# Evaluación
accuracy, f1 = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.6614
F1 Score: 0.6562


## 3.4 Transformer Encoder
Use nn.TransformerEncoder o transformers de Hugging Face con un modelo preentrenado como distilBERT (BERT)  
Para cada uno de los puntos anteriormente indicados puede usar los siguientes hiperparámetros:  
* Función de pérdida: nn.BCEWithLogitsLoss() o CrossEntropyLoss().
* Métricas: accuracy, F1 score.  

Entrenar cada modelo por separado y comparar desempeño.


### Opción A: nn.TransformerEncoder desde cero

In [24]:
import torch.nn as nn

class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, hidden_dim, num_layers, output_dim, max_len=200):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, embedding_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=0.1)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x) + self.pos_embedding[:, :x.size(1), :]  # (batch, seq_len, embed_dim)
        embedded = embedded.permute(1, 0, 2)  # (seq_len, batch, embed_dim)
        encoded = self.transformer(embedded)  # (seq_len, batch, embed_dim)
        out = encoded.mean(dim=0)             # Global average pooling
        return self.fc(out)

In [25]:
model = TransformerClassifier(vocab_size=len(vocab), embedding_dim=128, num_heads=4, hidden_dim=256, num_layers=2, output_dim=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

/opt/conda/lib/python3.11/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [26]:
for epoch in range(5):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.6527
Epoch 2, Loss: 0.5466
Epoch 3, Loss: 0.4897
Epoch 4, Loss: 0.4603
Epoch 5, Loss: 0.4284


In [27]:
# Evaluación
accuracy, f1 = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.7824
F1 Score: 0.7822


### Opción B: Modelo preentrenado distilBERT con Hugging Face

In [2]:
!pip install transformers

In [1]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

In [2]:
import numpy as np
from datasets import load_dataset

In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
dataset = load_dataset("stanfordnlp/imdb")

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [5]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds)
    }
# size 8 y epoch 1 para entrenar en local para terminar en aproximadamente 1,5 horas con cpu
training_args = TrainingArguments(
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'].shuffle(seed=42).select(range(10000)),  
    eval_dataset=tokenized_dataset['test'].select(range(2000)),
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.323400,0.259770,0.906000,0.000000


/opt/conda/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1250, training_loss=0.3698001525878906, metrics={'train_runtime': 5271.165, 'train_samples_per_second': 1.897, 'train_steps_per_second': 0.237, 'total_flos': 662336993280000.0, 'train_loss': 0.3698001525878906, 'epoch': 1.0})

# 📝¿Puede hacer un resumen de cada una de las 3 arquitecturas?

## 1. RNN (Redes Neuronales Recurrentes)
Idea principal: Las RNN procesan secuencias palabra por palabra, manteniendo un estado oculto que se actualiza en cada paso.

Ventajas: Simples y útiles para tareas secuenciales.

Limitaciones: Sufren de problemas de gradiente (desvanecimiento/explosión) en secuencias largas, lo que limita su capacidad de retención a largo plazo.

## 2. LSTM (Long Short-Term Memory)
Idea principal: Extensión de RNN diseñada para recordar información durante períodos más largos.

Componentes clave: Celdas de memoria, puertas de entrada, olvido y salida.

Ventajas: Maneja secuencias largas de manera más efectiva gracias a su estructura de puertas que controla el flujo de información.

## 3. Transformer (Encoder)
Idea principal: Utiliza mecanismos de atención para procesar todo el texto a la vez (paralelo, no secuencial).

Ventajas:

Más rápido de entrenar (paralelización).

Captura relaciones globales en la secuencia.

Limitaciones: Necesita más recursos computacionales (GPU recomendado).

# 📝¿Cuál de los 3 modelos clasifica mejor? Puede usar las Metricas solicitadas.

Comparación esperada:  
RNN  
Accuracy: 0.4991  
F1 Score: 0.4879  
LSTM  
Accuracy: 0.6614  
F1 Score: 0.6562  
TransformeEncoder  
Accuracy: 0.7824  
F1 Score: 0.7822  
¿Por qué gana el TransformerEncoder?  
Atención global (Self-Attention), El TransformerEncoder utiliza el mecanismo de self-attention, que le permite:  
Evaluar todas las palabras del texto a la vez.  
Identificar relaciones entre palabras lejanas, sin importar la distancia entre ellas.  
En cambio, RNN y LSTM procesan secuencialmente, lo que dificulta la captura de relaciones 
largo plazo.  largo plazo.  os.

### Modelo preentrenado distilBERT  
La precisión es alta, pero el F1-score es 0, lo cual es una fuerte señal de desequilibrio en las predicciones o en las clases
Esto puede ser por un desequilibrio de clases, **o muy probablemente por que solo hubo una epoca de entrenamiento debido a limitaciones de hardware**

# 📝 ¿Tarda más con CPU?

Sí.
Los modelos, especialmente LSTM y Transformer, hacen muchos cálculos matriciales.

Con CPU, estos cálculos son secuenciales y lentos.

Con GPU, se pueden paralelizar y acelerar drásticamente.

Conclusión: Usar GPU reduce drásticamente el tiempo de entrenamiento y evaluación.

# 📝¿Qué ventaja ofrece LSTM sobre RNN en el manejo de secuencias largas y por qué?

Problema del RNN: Al entrenar con secuencias largas, el gradiente se diluye (se hace muy pequeño), perdiendo contexto anterior.

LSTM soluciona esto: Usa una celda de memoria que puede "recordar" información útil durante muchas iteraciones.

Resultado: Mayor rendimiento en tareas donde el contexto lejano es importante, como texto largo o diálogos.

# 📝 ¿Cómo decide un Transformer a qué palabras prestar más atención al generar una representación y qué papel cumplen las matrices Q, K y V en ese proceso?  
Atención (Self-Attention)
Cada palabra "decide" a qué otras palabras prestar atención a través de:

Q (Query): Qué estoy buscando.

K (Key): Qué ofrece cada palabra.

V (Value): Información real de cada palabra.

Proceso:
Para cada par de palabras, se calcula una puntuación: Q ⋅ K.

Se aplica softmax para normalizar estas puntuaciones → pesos de atención.

Se multiplica por V para obtener una nueva representación ponderada.

Resultado: El modelo puede capturar relaciones de la frase "La película fue sorprendentemente buena" como:

"sorprendentemente buena” → atención sobre "sorprendentemente" modifica el significado de "buena”.